In [4]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
#Loading the Data 2.20.2018

xl = pd.ExcelFile("positions_20180220.xls")
df = xl.parse("Sheet1")
df= df[:-1]  #removing the irrelevant last row 


In [22]:
#Handling the Data

df_c =df[df.Issuer != 'FX' ].copy() #dropping FX
df_c= df_c.dropna(subset=['Issuer','Country','Exposure'])  #dropping Nan Values first from these two columns
idx = df_c['Exposure'] !=0 #Remove 0 Exposure
df_c = df_c.loc[idx,:]
idc=df_c['MTD P/L Price G/L'] !=0 #Remove 0 PNL
df_c =df_c.loc[idc,:]
##CASE 1 JABCAP EMEA
JABCAP_EMEA_df_original = df_c[df_c.FundCode =='JABCAP EMEA']
JABCAP_EMEA_df = JABCAP_EMEA_df_original.groupby('Issuer').sum()
JABCAP_EMEA_df.head()


,Fund ID,Exposure,DTD P/L Price G/L,MTD P/L Price G/L,YTD P/L Price G/L,Position
Issuer,,,,,,
ANTA Sports Products Ltd,6.0,0.002650,0.0,-25929.5903,-23295.1687,129164.0
Al Hammadi Development and Investment Co,12.0,0.003554,0.0,-65558.4659,-28382.1159,83824.0
Al Rajhi Bank,6.0,0.027717,0.0,97244.8379,626884.7576,318878.0
AngloGold Ashanti Ltd,6.0,0.004673,0.0,-224167.3965,-78015.6061,103382.0
Banco do Brasil SA,6.0,0.014411,0.0,-4996.9831,50798.5591,271823.0


In [23]:
###ATTRIBUTES

###NUMERICAL VALUES
## LONG-SHORT: functions + binary values
JABCAP_EMEA_df['Short']=JABCAP_EMEA_df['Exposure'].apply(lambda x : 1 if x <0 else 0)
JABCAP_EMEA_df['Long']=JABCAP_EMEA_df['Exposure'].apply(lambda x : 1 if x >0 else 0)


##Profit and Loss
## Profit-Loss (based on Monthly Data)
JABCAP_EMEA_df['MTD P/L Price']=JABCAP_EMEA_df['MTD P/L Price G/L'].apply(lambda x : 1 if x > 0 else 0)
JABCAP_EMEA_df['YTD P/L Price']=JABCAP_EMEA_df['YTD P/L Price G/L'].apply(lambda x : 1 if x > 0 else 0)

JC = JABCAP_EMEA_df.loc[:,['Short','Long','MTD P/L Price','YTD P/L Price']]
####STRING VALUES
##REGION : AMERICAN-EUROPEAN
#Defining Functions 
American = {'United States', 'Canada'}
European  ={'Austria', 'Belgium', 'Denmark', 'EMU', 'Finland', 'France', 'Germany', 'Hungary', 'Ireland', 'Italy', 
            'Japan', 'Jersey', 'Luxembourg', 'Netherlands', 'Norway', 'Poland', 'Portugal', 'Spain', 'Sweden',
            'Switzerland', 'United Kingdom','Czech Republic','Cyprus','Greece'}
Other = set(df_c.Country) - American - European

# def get_region(country):
#     Region = dict()
#     for i in American:
#       Region[i]='Americas'
#     for i in European:
#       Region[i]='Europe'
#     try:
#         return Region[country]
#     except:
#         return 'Other'
    
def american(country):
    if country in American:
        return 1
    else:
        return 0
def european(country):
    if country in European:
        return 1
    else:
        return 0   
#JABCAP_EMEA_df_original['Region']= JABCAP_EMEA_df_original['Country'].apply(lambda x: get_region(x))

#Filling the binary values
JABCAP_EMEA_df_original['American']= JABCAP_EMEA_df_original['Country'].apply(lambda x: american(x))
JABCAP_EMEA_df_original['European']= JABCAP_EMEA_df_original['Country'].apply(lambda x: european(x))
JCC = JABCAP_EMEA_df_original.loc[:,['Issuer','American','European']]           
JCC.set_index('Issuer', inplace= True)
JCC=JCC[~JCC.index.duplicated(keep='first')]

JC.head(50)

C:\Users\charbel.abboud\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\charbel.abboud\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Short,Long,MTD P/L Price,YTD P/L Price
Issuer,,,,
ANTA Sports Products Ltd,0,1,0,0
Al Hammadi Development and Investment Co,0,1,0,0
Al Rajhi Bank,0,1,1,1
AngloGold Ashanti Ltd,0,1,0,0
Banco do Brasil SA,0,1,0,1
Bank Al-Jazira,0,1,0,0
Bank of China Ltd,0,1,0,1
China Literature Ltd,0,1,0,0
China Mengniu Dairy Co Ltd,0,1,1,1


In [34]:
# Concatenating the dataframes
J = pd.concat([JC, JCC],axis=1)
#J.drop_duplicates().reset_index(drop=True)
#J.fillna(0, inplace=True)


In [35]:
###Adjencancy Matrix

##Creating it
nbr = len(J.index)
A = pd.DataFrame(np.zeros([nbr,nbr]))
A.index = J.index
A.columns = J.index
import itertools
from sklearn.metrics import jaccard_similarity_score
for col in itertools.combinations(J.index, 2):
    Y1 = J.loc[col[0],:]
    Y2 = J.loc[col[1],:]
    score = jaccard_similarity_score(Y1,Y2)
    A.loc[col[0],col[1]] = score
    
#from sklearn.metrics.pairwise import pairwise_distances
#A_M = 1 - pairwise_distances(JC, metric = "hamming")
#A_M = pd.DataFrame(A_M, index=JC.index, columns=JC.index)

##Making it symmetric
for i in range(0,len(A.index)):
    for j in range(i, len(A.index)):
        A.values[j][i] = A.values[i][j]
        
##Exporting it
import csv
A.to_csv('A.csv')

In [36]:
A.head()

,ANTA Sports Products Ltd,Al Hammadi Development and Investment Co,Al Rajhi Bank,AngloGold Ashanti Ltd,Banco do Brasil SA,Bank Al-Jazira,Bank of China Ltd,China Literature Ltd,China Mengniu Dairy Co Ltd,China Resources Beer Holdings Co Ltd,...,TCS Group Holding PLC,Turk Telekomunikasyon AS,TurkDEX ISE 30 Futures,Turkcell Iletisim Hizmetleri AS,Turkiye Sise ve Cam Fabrikalari AS,UNITED STATES GOVERNMENT,Union National Bank PJSC,Wuliangye Yibin Co Ltd,Yanbu National Petrochemical Co,Yandex NV
ANTA Sports Products Ltd,0.000000,1.000000,0.666667,0.833333,0.833333,1.000000,0.833333,1.000000,0.666667,0.666667,...,0.666667,1.000000,0.666667,0.833333,1.000000,0.500000,1.000000,1.000000,0.833333,0.500000
Al Hammadi Development and Investment Co,1.000000,0.000000,0.666667,0.833333,0.833333,1.000000,0.833333,1.000000,0.666667,0.666667,...,0.666667,1.000000,0.666667,0.833333,1.000000,0.500000,1.000000,1.000000,0.833333,0.500000
Al Rajhi Bank,0.666667,0.666667,0.000000,0.500000,0.833333,0.666667,0.833333,0.666667,1.000000,1.000000,...,0.666667,0.666667,0.333333,0.833333,0.666667,0.833333,0.666667,0.666667,0.833333,0.833333
AngloGold Ashanti Ltd,0.833333,0.833333,0.500000,0.000000,0.666667,0.833333,0.666667,0.833333,0.500000,0.500000,...,0.500000,0.833333,0.500000,0.666667,0.833333,0.666667,0.833333,0.833333,0.666667,0.666667
Banco do Brasil SA,0.833333,0.833333,0.833333,0.666667,0.000000,0.833333,1.000000,0.833333,0.833333,0.833333,...,0.833333,0.833333,0.500000,1.000000,0.833333,0.666667,0.833333,0.833333,1.000000,0.666667
